# The Battle of Neighborhoods (Week 1)

### *Introduction*
A new startup after garnering much interest in its intial seed investment has decided to setup its headquarters. The company has thus decided to identify key factors that will help in deciding where to setup.  

### *Data Used*
1. City of Toronto - https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
2. Geographical coordinates of each postal code - http://cocl.us/Geospatial_data 

### *Library Imports*

In [176]:
#pip install requests BeautifulSoup4
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
import requests
import folium

import matplotlib.pyplot as plt # plotting library
import matplotlib.cm as cm
import matplotlib.colors as colors
# backend for rendering plots within the browser
%matplotlib inline 

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs
from geopy.geocoders import Nominatim
from selenium import webdriver
from bs4 import BeautifulSoup

pd.set_option("display.max_rows", None, "display.max_columns", None)

print('Libraries imported.')

Libraries imported.


In [294]:
results = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
#print(results.status_code)
#print(results.headers)
src=results.content
soup = BeautifulSoup(src,'lxml')
table = soup.find("table",attrs={"class":"wikitable"})
#table_header = table.find_all("th")
table_body = table.find_all("td")

pc=[]
bor=[]
nb=[]
for i in range(0,len(table_body),3):
    pc.append((table_body[i].text).strip())
    bor.append((table_body[i+1].text).strip())
    nb.append((table_body[i+2].text).strip())
    #print(table_body[i].text)
#print(pc)
#print(bor)
#print(nb)

Toronto = (pd.DataFrame({'Postal Code':pc,'Borough':bor,'Neighborhood':nb})).reset_index(drop=True)
# Filter all unassigned neighbourhoods
#Toronto = Toronto[(Toronto.Borough.notnull())]
#Toronto = Toronto[(Toronto.Borough != "Not assigned")]
Toronto['Neighborhood'] = np.where(Toronto['Neighborhood'] == 'Not assigned', Toronto['Borough'], Toronto['Neighborhood'])
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
Toronto.loc[ (Toronto.Neighborhood.isnull() == True) |
               (Toronto.Neighborhood == "Not assigned")
               , 'Neighborhood'] = Toronto.Borough

geo = pd.read_csv('Geospatial_Coordinates.csv')
Toronto = pd.merge(left=Toronto, right=geo, left_on='Postal Code', right_on='Postal Code')



# Group by PostalCode/Borough
#Toronto = pd.DataFrame(Toronto.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(list)).reset_index()
#Toronto['Neighborhood'] = Toronto['Neighborhood'].apply(lambda x: ', '.join(x))

In [295]:
Toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [296]:
Toronto.Borough.unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [297]:
Toronto.Neighborhood.unique()

array(['Parkwoods', 'Victoria Village', 'Regent Park, Harbourfront',
       'Lawrence Manor, Lawrence Heights',
       "Queen's Park, Ontario Provincial Government",
       'Islington Avenue, Humber Valley Village', 'Malvern, Rouge',
       'Don Mills', 'Parkview Hill, Woodbine Gardens',
       'Garden District, Ryerson', 'Glencairn',
       'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale',
       'Rouge Hill, Port Union, Highland Creek', 'Woodbine Heights',
       'St. James Town', 'Humewood-Cedarvale',
       'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood',
       'Guildwood, Morningside, West Hill', 'The Beaches', 'Berczy Park',
       'Caledonia-Fairbanks', 'Woburn', 'Leaside', 'Central Bay Street',
       'Christie', 'Cedarbrae', 'Hillcrest Village',
       'Bathurst Manor, Wilson Heights, Downsview North',
       'Thorncliffe Park', 'Richmond, Adelaide, King',
       'Dufferin, Dovercourt Village', 'Scarborough Village',
       'Fairview, H

In [298]:
Toronto['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [299]:
Toronto['Neighborhood'].value_counts()

Downsview                                                                                                                                 4
Don Mills                                                                                                                                 2
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport                                1
Canada Post Gateway Processing Centre                                                                                                     1
Willowdale, Willowdale West                                                                                                               1
Runnymede, Swansea                                                                                                                        1
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto                                                      1
University of Toront

### *Toronto Map higlighting all the neigborhoods*

In [300]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Borough'], Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


### *Explore neighborhoods in Toronto with Foursquare API*

In [301]:
CLIENT_ID = 'TJQFNQLD13APOGBQ0IZUQEHVLUX4UQVIGYWRO4CYRVHOWBV4' # your Foursquare ID
CLIENT_SECRET = 'PECTIYU42AT2IKKGL4DG2KZGCKW5LC1VPTLVGMNAFUVJQCYT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TJQFNQLD13APOGBQ0IZUQEHVLUX4UQVIGYWRO4CYRVHOWBV4
CLIENT_SECRET:PECTIYU42AT2IKKGL4DG2KZGCKW5LC1VPTLVGMNAFUVJQCYT


In [302]:
#Toronto = Toronto[Toronto['Borough'] == 'North York'].reset_index(drop=True)
Toronto = Toronto[Toronto['Borough'] == 'Scarborough'].reset_index(drop=True)

LIMIT = 100 
radius = 500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=Toronto['Neighborhood'],
                                   latitudes=Toronto['Latitude'],
                                   longitudes=Toronto['Longitude']
                                  )
toronto_venues.groupby('Neighborhood').count()

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",5,5,5,5,5,5
"Golden Mile, Clairlea, Oakridge",9,9,9,9,9,9
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",4,4,4,4,4,4


In [303]:
address = 'Scarborough, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Borough'], Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto, Ontario are 43.773077, -79.257774.


In [304]:
#print(toronto_venues.shape)
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


In [305]:
#Venues per Neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",5,5,5,5,5,5
"Golden Mile, Clairlea, Oakridge",9,9,9,9,9,9
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",4,4,4,4,4,4


In [306]:
print('There are {} distinct venues in {} categories.'.format(
    len(toronto_venues['Venue'].unique()),len(toronto_venues['Venue Category'].unique())))

There are 77 distinct venues in 55 categories.


In [307]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
neighb= toronto_venues['Neighborhood'] 
#toronto_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
# move neighborhood column to the first column
toronto_onehot.insert(0, 'Neighborhood', neighb)
toronto_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Burger Joint,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [308]:
toronto_onehot_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_onehot_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Burger Joint,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.25,0.000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.25,0.000000,0.000000,0.000000,0.0
1,"Birch Cliff, Cliffside West",0.0,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000,0.000000,0.000000,0.25,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.25,0.000000,0.000000,0.000000,0.0
2,Cedarbrae,0.0,0.125,0.0,0.125000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.125,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000,0.000000,0.125000,0.125000,0.00,0.000000,0.000000,0.125,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.00,0.000000,0.125000,0.000000,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000,0.0,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.083333,0.000000,0.00,0.083333,0.00,0.000000,0.000,0.083333,0.083333,0.083333,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.0,0.000000,0.083333,0.000000,0.00,0.00,0.000,0.000000,0.000,0.0,0.0,0.000000,0.083333,0.000000,0.0,0.083333,0.166667,0.0,0.000,0.000,0.000000,0.00,0.000000,0.083333,0.000000,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.5,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000,0.0,0.5,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.0
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.200000,0.000000,0.00,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.4,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.2
6,"Golden Mile, Clairlea, Oakridge",0.0,0.000,0.0,0.222222,0.000000,0.000000,0.000000,0.000000,0.111111,0.111111,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.111111,0.0,0.111111,0.000000,0.000000,0.00,0.00,0.000,0.111111,0.000,0.0,0.0,0.000000,0.000000,0.111111,0.0,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.00,0.111111,0.000000,0.000000,0.0
7,"Guildwood, Morningside, West Hill",0.0,0.000,0.0,0.000000,0.125000,0.000000,0.125000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.000000

In [309]:
num_top_venues = 10

for hood in toronto_onehot_grouped['Neighborhood']:
    #print("----"+hood+"----")
    temp = toronto_onehot_grouped[toronto_onehot_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

                       venue  freq
0               Skating Rink  0.25
1             Breakfast Spot  0.25
2  Latin American Restaurant  0.25
3                     Lounge  0.25
4        American Restaurant  0.00
5               Noodle House  0.00
6               Intersection  0.00
7         Italian Restaurant  0.00
8          Korean Restaurant  0.00
9             Medical Center  0.00


                       venue  freq
0            College Stadium  0.25
1               Skating Rink  0.25
2      General Entertainment  0.25
3                       Café  0.25
4               Noodle House  0.00
5         Italian Restaurant  0.00
6          Korean Restaurant  0.00
7  Latin American Restaurant  0.00
8                     Lounge  0.00
9             Medical Center  0.00


                       venue  freq
0       Caribbean Restaurant  0.12
1        Fried Chicken Joint  0.12
2                     Bakery  0.12
3                       Bank  0.12
4            Thai Restaurant  0.12
5               

In [310]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_onehot_grouped['Neighborhood']

for ind in np.arange(toronto_onehot_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_onehot_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
1,"Birch Cliff, Cliffside West",Skating Rink,General Entertainment,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Gym,Grocery Store,Gas Station,Fried Chicken Joint
2,Cedarbrae,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Gym
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Fast Food Restaurant,Convenience Store,Noodle House,Italian Restaurant,Pharmacy,Chinese Restaurant,Bank,Fried Chicken Joint,Gas Station
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Hobby Shop,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Bakery,Bank,Grocery Store,Athletics & Sports,General Entertainment,Gas Station
6,"Golden Mile, Clairlea, Oakridge",Bakery,Intersection,Bus Station,Soccer Field,Ice Cream Shop,Metro Station,Park,Bus Line,Department Store,Discount Store
7,"Guildwood, Morningside, West Hill",Medical Center,Bank,Restaurant,Rental Car Location,Mexican Restaurant,Breakfast Spot,Intersection,Electronics Store,Vietnamese Restaurant,Discount Store
8,"Kennedy Park, Ionview, East Birchmount Park",Bus Station,Department Store,Hobby Shop,Coffee Shop,Vietnamese Restaurant,College Stadium,Gym,Grocery Store,General Entertainment,Gas Station
9,"Malvern, Rouge",Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store


In [311]:
neighborhoods_venues_sorted.iloc[11,]

Neighborhood              Rouge Hill, Port Union, Highland Creek
1st Most Common Venue                               Home Service
2nd Most Common Venue                                        Bar
3rd Most Common Venue                              Moving Target
4th Most Common Venue                                Auto Garage
5th Most Common Venue                            College Stadium
6th Most Common Venue                                        Gym
7th Most Common Venue                              Grocery Store
8th Most Common Venue                      General Entertainment
9th Most Common Venue                                Gas Station
10th Most Common Venue                       Fried Chicken Joint
Name: 11, dtype: object

### *Using K means to cluster each neighborhood*

In [358]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_onehot_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(toronto_grouped_clustering)

toronto_merged = Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# Check for neighborhoods that have NAN
toronto_merged = toronto_merged.dropna()

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

print(toronto_merged.shape,len(kmeans.labels_))
#toronto_merged.head()

(16, 16) 16


In [359]:
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    # print list 
    for x in unique_list: 
        print (x)
    
unique(kmeans.labels_)

0
4
3
2
1


In [360]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### *Examining Each Cluster*

In [361]:
cluster_0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]
cluster_1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]
cluster_2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]
cluster_3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]
cluster_4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

In [362]:
cluster_0

,Borough,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Scarborough,-79.194353,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,0
1,Scarborough,-79.160497,Home Service,Bar,Moving Target,Auto Garage,College Stadium,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,0
2,Scarborough,-79.188711,Medical Center,Bank,Restaurant,Rental Car Location,Mexican Restaurant,Breakfast Spot,Intersection,Electronics Store,Vietnamese Restaurant,Discount Store,0
3,Scarborough,-79.216917,Coffee Shop,Korean Restaurant,Hobby Shop,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,0
5,Scarborough,-79.239476,Playground,Vietnamese Restaurant,Coffee Shop,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,0
6,Scarborough,-79.262029,Bus Station,Department Store,Hobby Shop,Coffee Shop,Vietnamese Restaurant,College Stadium,Gym,Grocery Store,General Entertainment,Gas Station,0
7,Scarborough,-79.284577,Bakery,Intersection,Bus Station,Soccer Field,Ice Cream Shop,Metro Station,Park,Bus Line,Department Store,Discount Store,0
11,Scarborough,-79.295849,Middle Eastern Restaurant,Auto Garage,Bakery,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,0
13,Scarborough,-79.304302,Pizza Place,Fast Food Restaurant,Convenience Store,Noodle House,Italian Restaurant,Pharmacy,Chinese Restaurant,Bank,Fried Chicken Joint,Gas Station,0
14,Scarborough,-79.284577,Playground,Park,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,0


In [363]:
cluster_1

,Borough,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
10,Scarborough,-79.273304,Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Bakery,Bank,Grocery Store,Athletics & Sports,General Entertainment,Gas Station,1
12,Scarborough,-79.262029,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,1


In [364]:
cluster_2

,Borough,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
9,Scarborough,-79.264848,Skating Rink,General Entertainment,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Gym,Grocery Store,Gas Station,Fried Chicken Joint,2


In [365]:
cluster_3

,Borough,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
8,Scarborough,-79.239476,American Restaurant,Motel,Hobby Shop,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,3
15,Scarborough,-79.318389,Chinese Restaurant,Fast Food Restaurant,Grocery Store,Pharmacy,Coffee Shop,Discount Store,Gym,Burger Joint,Pizza Place,Breakfast Spot,3


In [368]:
cluster_4

,Borough,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
4,Scarborough,-79.239476,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Gym,4
